Itay Koren

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
import psycopg2

In [2]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-sales-data-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = reflection.Inspector.from_engine(engine)


<ipython-input-2-60b087478d5e>:14: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = reflection.Inspector.from_engine(engine)


In [3]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

rep_sales table:

        OrderDate: date when the order was placed

        Region: geographical area in which the sale was made.

        Rep: sales representative's name

        Item: name of the item sold

        Units: number of units sold

        UnitCost: cost of one unit

        Total: total cost of the order - Units x UnitCost

Item table:

        Item: name of the item

        MinPrice: minimum price that the item can be sold.

1. What region had the most units sold for pencil?  


In [4]:
query = '''
select 
    "Region",
    sum("Units")
    
from 
    rep_sales
where
    "Item" = 'Pencil'
group by
    "Region"
order by
    sum desc
limit 1;
'''

queryResult(query)

,Region,sum
0,Central,498



2. For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. How many transactions weren’t calculated for commissions?


In [5]:
query = '''
select 
    count("Rep")
from 
    rep_sales
    left join item
    on item."Item" = rep_sales."Item"
where 
    "MinPrice" >= "Unit_Cost"

'''

queryResult(query)

,count
0,30



3. For West region what was the share of each sales rep in the data?


In [6]:
query = '''
select
    "Rep",
    100 * rep_total_sales / sum(rep_total_sales) over() as rep_share
from
    (
    select 
        "Rep",
        sum("Units" * "Unit_Cost") as rep_total_sales
    from 
        rep_sales
    where 
        "Region" = 'West'
    group by 
        "Rep"
    ) as subq
group by 
    "Rep", rep_total_sales
'''

queryResult(query)

,Rep,rep_share
0,Sorvino,38.585115
1,Thompson,61.414885



4. For the item Binder what was the share of each sales rep in the data?


In [7]:
query = '''
select
    rep_sales."Rep",
    sum("Total"),
    100 * sum("Total") / binder_total as binder_share
from
    rep_sales 
    left join (
    select
        distinct "Rep",
        sum("Total") over() as binder_total
    from
        rep_sales    
    where
        "Item" = 'Binder'
    ) as subq on subq."Rep" = rep_sales."Rep"
    
where
    "Item" = 'Binder'
group by 
    rep_sales."Rep", binder_total
'''

queryResult(query)

,Rep,sum,binder_share
0,Morgan,251.72,3.058893
1,Andrews,139.72,1.697873
2,Joe,347.71,4.225361
3,Gill,1132.74,13.765020
4,Sorvino,139.93,1.700425
5,Jones,1386.52,16.848946
6,Smith,952.00,11.568673
7,Thompson,832.00,10.110437
8,Howard,57.71,0.701290
9,Jardine,1054.09,12.809268



5. For Jardine what was the share of each item sold in the data?


In [8]:
query = '''
select
    rep_sales."Item",
    sum(rep_sales."Total") as item_total_jardine,
    100 * sum(rep_sales."Total") / item_total_all as jardine_share
from
    rep_sales
    left join(
    select
        "Item",
        sum("Total") as item_total_all
    from
        rep_sales
    group by
        "Item"
    ) as subq on subq."Item" = rep_sales."Item"
where
    "Rep" = 'Jardine'
group by
    rep_sales."Item", item_total_all
'''

queryResult(query)

,Item,item_total_jardine,jardine_share
0,Pen Set,249.50,5.327031
1,Marker Set,983.18,14.854556
2,Binder,1054.09,12.809268
3,Pencil,628.74,26.863950
4,Pen,91.29,3.296834



6. Which item is the most profitable to sell? Get the percentage of the diff from unit price to minimum and rank it for the entire data


In [60]:
query = '''
select
   rep_sales."Item",
   "MinPrice",
    max("Unit_Cost") as max_unit_cost,
    max("Unit_Cost") - "MinPrice" as max_min_dif,
    100 * (max("Unit_Cost") - "MinPrice")/ "MinPrice" as precentage_diff
from
    rep_sales
    left join
    (select 
        distinct "Item", 
        "MinPrice" 
    from 
        item) as min_price_t
    on rep_sales."Item" = min_price_t."Item"
group by
    rep_sales."Item", "MinPrice"
order by precentage_diff desc
'''

queryResult(query)

,Item,MinPrice,max_unit_cost,max_min_dif,precentage_diff
0,Pen,1.75,19.99,18.24,1042.285714
1,Binder,1.99,21.99,20.00,1005.025126
2,Pencil,1.29,4.99,3.70,286.821705
3,Pen Set,4.99,17.00,12.01,240.681363
4,Marker Set,5.99,14.99,9.00,150.250417
5,Desk,125.00,275.00,150.00,120.000000



7. Which sales rep had the highest price difference from the minimum price on these items combined: pen, pen set and pencil?

In [58]:
query = '''

select 
    "Item" ,
    sum("MinPrice") as items_total_sum
from 
    item
where
    "Item" = 'Pen' or "Item" = 'Pen Set' or "Item" = 'Pencil'

'''

query = '''
select 
    "Rep",
    "Item",
    max("Unit_Cost"),
    rank() over(partition by "Rep" order by "Item") as rank
from 
    rep_sales
where
    "Item" = 'Pen' 
    or "Item" = 'Pen Set' 
    or "Item" = 'Pencil'
group by 
    "Rep", "Item"
order by 
    "Rep";
'''
# query = '''
# select 
#     "Rep",
#     "Item",
#     "Unit_Cost"
# from 
#     rep_sales
# where
#     "Item" = 'Pen' 
#     or "Item" = 'Pen Set' 
#     or "Item" = 'Pencil'
# '''


queryResult(query)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "rank" does not exist
LINE 13:     and rank = 3
                 ^

[SQL: 
select 
    "Rep",
    "Item",
    max("Unit_Cost"),
    rank() over(partition by "Rep" order by "Item") as rank
from 
    rep_sales
where
    ("Item" = 'Pen' 
    or "Item" = 'Pen Set' 
    or "Item" = 'Pencil')
    and rank = 3
group by 
    "Rep", "Item"
order by 
    "Rep";
]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [56]:
# query = '''

# select 
#     "Item" ,
#     sum("MinPrice") as items_total_sum
# from 
#     item
# where
#     "Item" = 'Pen' or "Item" = 'Pen Set' or "Item" = 'Pencil'

# '''

# query = '''
# select 
#     "Rep",
#     "Item",
#     max("Unit_Cost")
# from 
#     rep_sales
# where
#     "Item" = 'Pen' 
#     or "Item" = 'Pen Set' 
#     or "Item" = 'Pencil'
# group by 
#     "Rep", "Item"
# order by 
#     "Rep"
# '''
query = '''
select  
distinct
    "Rep",
    "Item",
    "Unit_Cost"
from 
    rep_sales
where
    "Item" = 'Pen' 
    or "Item" = 'Pen Set' 
    or "Item" = 'Pencil'
order by
    "Rep"
'''


queryResult(query)

,Rep,Item,Unit_Cost
0,Andrews,Pen,1.50
1,Andrews,Pencil,1.29
2,Andrews,Pencil,1.99
3,Gill,Pen,1.49
4,Gill,Pen,19.99
5,Gill,Pencil,1.29
6,Howard,Pen,4.99
7,Jardine,Pen,1.79
8,Jardine,Pen Set,4.99
9,Jardine,Pencil,4.99
